# 🔹 Loan Approval Prediction ⭐⭐⭐⭐⭐

## Predict: Loan Approved (Yes/No)

### Features:
- **income**: Applicant's annual income
- **credit_score**: Credit score of the applicant
- **employment_years**: Years of employment
- **loan_amount**: Requested loan amount

### Models:
1. **Logistic Regression**
2. **Decision Tree**

### Pipeline:
1. Data Loading & Exploration
2. Data Cleaning & Preprocessing
3. Feature Engineering
4. Model Training & Evaluation
5. Model Export (joblib)


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix, 
    roc_auc_score,
    roc_curve,
    precision_recall_curve
)
from sklearn.pipeline import Pipeline
import joblib
import warnings
warnings.filterwarnings('ignore')

# Set style for plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)


## 1. Data Loading & Exploration


In [ ]:
# Create synthetic loan approval dataset
# In a real scenario, you would load from Kaggle or other sources
np.random.seed(42)

# Generate synthetic data
n_samples = 10000

# Generate features
income = np.random.normal(50000, 20000, n_samples)
income = np.clip(income, 20000, 150000)  # Clip to reasonable range

credit_score = np.random.normal(650, 100, n_samples)
credit_score = np.clip(credit_score, 300, 850)

employment_years = np.random.exponential(5, n_samples)
employment_years = np.clip(employment_years, 0, 40)

loan_amount = np.random.normal(30000, 20000, n_samples)
loan_amount = np.clip(loan_amount, 5000, 200000)

# Create loan-to-income ratio (important feature)
loan_to_income = loan_amount / income

# Generate target variable (loan_approved) based on business logic
# Higher income, credit score, employment years = higher approval chance
# Lower loan amount relative to income = higher approval chance
approval_prob = (
    0.3 * (income / 100000) +  # Income factor
    0.3 * (credit_score / 850) +  # Credit score factor
    0.2 * (employment_years / 20) +  # Employment stability
    0.2 * (1 - np.clip(loan_to_income, 0, 1))  # Loan-to-income ratio (inverse)
)

# Add some noise
approval_prob += np.random.normal(0, 0.1, n_samples)
approval_prob = np.clip(approval_prob, 0, 1)

# Convert to binary
loan_approved = (approval_prob > 0.5).astype(int)

# Create DataFrame
df = pd.DataFrame({
    'income': income,
    'credit_score': credit_score,
    'employment_years': employment_years,
    'loan_amount': loan_amount,
    'loan_approved': loan_approved
})

print("Dataset created successfully!")
print(f"Shape: {df.shape}")
print("\nFirst few rows:")
df.head()


In [ ]:
# Basic information about the dataset
print("=" * 50)
print("DATASET INFORMATION")
print("=" * 50)
print(f"\nDataset Shape: {df.shape}")
print(f"\nColumn Names: {df.columns.tolist()}")
print(f"\nData Types:\n{df.dtypes}")
print(f"\nMissing Values:\n{df.isnull().sum()}")
print(f"\nBasic Statistics:\n{df.describe()}")


In [ ]:
# Check target variable distribution
print("=" * 50)
print("TARGET VARIABLE DISTRIBUTION")
print("=" * 50)
print(f"\nLoan Approval Distribution:\n{df['loan_approved'].value_counts()}")
print(f"\nLoan Approval Percentage:\n{df['loan_approved'].value_counts(normalize=True) * 100}")

# Visualize target distribution
plt.figure(figsize=(8, 5))
df['loan_approved'].value_counts().plot(kind='bar', color=['#ff6b6b', '#51cf66'])
plt.title('Loan Approval Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Loan Approved (0=No, 1=Yes)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## 2. Exploratory Data Analysis (EDA)


In [ ]:
# Distribution of features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions', fontsize=16, fontweight='bold')

features = ['income', 'credit_score', 'employment_years', 'loan_amount']
colors = ['#4dabf7', '#51cf66', '#ffd43b', '#ff6b6b']

for idx, (feature, color) in enumerate(zip(features, colors)):
    ax = axes[idx // 2, idx % 2]
    df[feature].hist(bins=50, ax=ax, color=color, alpha=0.7, edgecolor='black')
    ax.set_title(f'{feature.replace("_", " ").title()}', fontsize=12, fontweight='bold')
    ax.set_xlabel(feature.replace("_", " ").title(), fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Box plots to check for outliers
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Feature Distributions by Loan Approval Status', fontsize=16, fontweight='bold')

features = ['income', 'credit_score', 'employment_years', 'loan_amount']
colors = ['#4dabf7', '#51cf66', '#ffd43b', '#ff6b6b']

for idx, (feature, color) in enumerate(zip(features, colors)):
    ax = axes[idx // 2, idx % 2]
    df.boxplot(column=feature, by='loan_approved', ax=ax, patch_artist=True,
               boxprops=dict(facecolor=color, alpha=0.7))
    ax.set_title(f'{feature.replace("_", " ").title()} by Approval Status', fontsize=12, fontweight='bold')
    ax.set_xlabel('Loan Approved (0=No, 1=Yes)', fontsize=10)
    ax.set_ylabel(feature.replace("_", " ").title(), fontsize=10)
    plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()


In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()


## 3. Data Cleaning & Preprocessing


In [ ]:
# Check for missing values
print("Missing values check:")
print(df.isnull().sum())
print(f"\nTotal missing values: {df.isnull().sum().sum()}")

# Check for duplicates
print(f"\nDuplicate rows: {df.duplicated().sum()}")

# Check for negative values in features that shouldn't have them
print("\nNegative values check:")
print(f"Income: {(df['income'] < 0).sum()}")
print(f"Credit Score: {(df['credit_score'] < 0).sum()}")
print(f"Employment Years: {(df['employment_years'] < 0).sum()}")
print(f"Loan Amount: {(df['loan_amount'] < 0).sum()}")


In [ ]:
# Handle any outliers using IQR method (optional, but good practice)
def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]
    return df_clean

# For this dataset, we'll keep all data as it's synthetic and already bounded
# But in real scenarios, you might want to remove outliers
print("Data cleaning complete. All values are within reasonable ranges.")


In [ ]:
# Feature Engineering: Create additional features
df_processed = df.copy()

# Loan to Income Ratio (important feature)
df_processed['loan_to_income_ratio'] = df_processed['loan_amount'] / df_processed['income']

# Debt Service Ratio (monthly loan payment / monthly income)
# Assuming 5% annual interest rate and 5-year term
monthly_interest_rate = 0.05 / 12
num_payments = 60  # 5 years
monthly_loan_payment = df_processed['loan_amount'] * (
    monthly_interest_rate * (1 + monthly_interest_rate)**num_payments
) / ((1 + monthly_interest_rate)**num_payments - 1)
monthly_income = df_processed['income'] / 12
df_processed['debt_service_ratio'] = monthly_loan_payment / monthly_income

# Credit Score Category (optional, can help with non-linear relationships)
df_processed['credit_score_category'] = pd.cut(
    df_processed['credit_score'],
    bins=[0, 580, 670, 740, 850],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

print("Feature engineering complete!")
print(f"\nNew features created:")
print(f"- loan_to_income_ratio")
print(f"- debt_service_ratio")
print(f"- credit_score_category")
print(f"\nUpdated shape: {df_processed.shape}")
df_processed.head()


In [ ]:
# Prepare features for modeling
# Select features for modeling
feature_columns = ['income', 'credit_score', 'employment_years', 'loan_amount', 
                   'loan_to_income_ratio', 'debt_service_ratio']

# One-hot encode credit_score_category if using it
# For simplicity, we'll use numeric features only
X = df_processed[feature_columns]
y = df_processed['loan_approved']

print("Features selected:")
print(feature_columns)
print(f"\nX shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nTarget distribution:\n{y.value_counts()}")


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data split complete!")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"\nTraining set target distribution:\n{y_train.value_counts()}")
print(f"\nTest set target distribution:\n{y_test.value_counts()}")


## 4. Model Training

### 4.1 Logistic Regression


In [ ]:
# Create pipeline with StandardScaler and Logistic Regression
lr_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(random_state=42, max_iter=1000))
])

# Train the model
print("Training Logistic Regression model...")
lr_pipeline.fit(X_train, y_train)

# Make predictions
y_pred_lr = lr_pipeline.predict(X_test)
y_pred_proba_lr = lr_pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
print("\n" + "="*50)
print("LOGISTIC REGRESSION RESULTS")
print("="*50)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred_lr)}")


In [ ]:
# Confusion Matrix for Logistic Regression
cm_lr = confusion_matrix(y_test, y_pred_lr)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', cbar=True,
            xticklabels=['Not Approved', 'Approved'],
            yticklabels=['Not Approved', 'Approved'])
plt.title('Logistic Regression - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# ROC Curve for Logistic Regression
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_pred_proba_lr)

plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, y_pred_proba_lr):.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Logistic Regression', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


### 4.2 Decision Tree


In [ ]:
# Create pipeline with StandardScaler and Decision Tree
# Note: Decision Tree doesn't require scaling, but we'll keep it for consistency
dt_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', DecisionTreeClassifier(random_state=42, max_depth=10, min_samples_split=20))
])

# Train the model
print("Training Decision Tree model...")
dt_pipeline.fit(X_train, y_train)

# Make predictions
y_pred_dt = dt_pipeline.predict(X_test)
y_pred_proba_dt = dt_pipeline.predict_proba(X_test)[:, 1]

# Evaluate the model
print("\n" + "="*50)
print("DECISION TREE RESULTS")
print("="*50)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred_dt):.4f}")
print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba_dt):.4f}")
print(f"\nClassification Report:\n{classification_report(y_test, y_pred_dt)}")


In [ ]:
# Confusion Matrix for Decision Tree
cm_dt = confusion_matrix(y_test, y_pred_dt)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Greens', cbar=True,
            xticklabels=['Not Approved', 'Approved'],
            yticklabels=['Not Approved', 'Approved'])
plt.title('Decision Tree - Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()


In [ ]:
# ROC Curve for Decision Tree
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, y_pred_proba_dt)

plt.figure(figsize=(8, 6))
plt.plot(fpr_dt, tpr_dt, label=f'Decision Tree (AUC = {roc_auc_score(y_test, y_pred_proba_dt):.4f})', linewidth=2)
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve - Decision Tree', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Feature Importance for Decision Tree
feature_importance = dt_pipeline.named_steps['classifier'].feature_importances_

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': feature_columns,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=importance_df, x='Importance', y='Feature', palette='viridis')
plt.title('Decision Tree - Feature Importance', fontsize=14, fontweight='bold')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.tight_layout()
plt.show()

print("\nFeature Importance:")
print(importance_df)


## 5. Model Comparison


In [ ]:
# Compare both models
comparison_df = pd.DataFrame({
    'Model': ['Logistic Regression', 'Decision Tree'],
    'Accuracy': [
        accuracy_score(y_test, y_pred_lr),
        accuracy_score(y_test, y_pred_dt)
    ],
    'ROC-AUC': [
        roc_auc_score(y_test, y_pred_proba_lr),
        roc_auc_score(y_test, y_pred_proba_dt)
    ]
})

print("="*50)
print("MODEL COMPARISON")
print("="*50)
print(comparison_df.to_string(index=False))

# Visualize comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
            color=['#4dabf7', '#51cf66'], alpha=0.7, edgecolor='black')
axes[0].set_title('Model Accuracy Comparison', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_ylim([0, 1])
axes[0].grid(axis='y', alpha=0.3)
for i, v in enumerate(comparison_df['Accuracy']):
    axes[0].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

# ROC-AUC comparison
axes[1].bar(comparison_df['Model'], comparison_df['ROC-AUC'], 
            color=['#4dabf7', '#51cf66'], alpha=0.7, edgecolor='black')
axes[1].set_title('Model ROC-AUC Comparison', fontsize=12, fontweight='bold')
axes[1].set_ylabel('ROC-AUC Score', fontsize=11)
axes[1].set_ylim([0, 1])
axes[1].grid(axis='y', alpha=0.3)
for i, v in enumerate(comparison_df['ROC-AUC']):
    axes[1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Combined ROC Curve
plt.figure(figsize=(10, 7))
plt.plot(fpr_lr, tpr_lr, label=f'Logistic Regression (AUC = {roc_auc_score(y_test, y_pred_proba_lr):.4f})', 
         linewidth=2, color='#4dabf7')
plt.plot(fpr_dt, tpr_dt, label=f'Decision Tree (AUC = {roc_auc_score(y_test, y_pred_proba_dt):.4f})', 
         linewidth=2, color='#51cf66')
plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier', linewidth=1.5)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve Comparison', fontsize=14, fontweight='bold')
plt.legend(loc='lower right', fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Export both models using joblib
import os

# Create model directory if it doesn't exist
os.makedirs('Backend/model', exist_ok=True)

# Export Logistic Regression model
joblib.dump(lr_pipeline, 'Backend/model/loan_approval_lr.pkl')
print("✓ Logistic Regression model exported to 'Backend/model/loan_approval_lr.pkl'")

# Export Decision Tree model
joblib.dump(dt_pipeline, 'Backend/model/loan_approval_dt.pkl')
print("✓ Decision Tree model exported to 'Backend/model/loan_approval_dt.pkl'")

# Also export feature names for reference
import json
model_info = {
    'feature_names': feature_columns,
    'target_name': 'loan_approved',
    'model_types': ['logistic_regression', 'decision_tree']
}

with open('Backend/model/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)
print("✓ Model info exported to 'Backend/model/model_info.json'")

print("\n" + "="*50)
print("MODEL EXPORT COMPLETE!")
print("="*50)


In [ ]:
# Verify the exported models can be loaded
print("Verifying exported models...")

# Load and test Logistic Regression
lr_loaded = joblib.load('Backend/model/loan_approval_lr.pkl')
test_pred_lr = lr_loaded.predict(X_test[:5])
print(f"✓ Logistic Regression model loaded successfully")
print(f"  Sample predictions: {test_pred_lr}")

# Load and test Decision Tree
dt_loaded = joblib.load('Backend/model/loan_approval_dt.pkl')
test_pred_dt = dt_loaded.predict(X_test[:5])
print(f"✓ Decision Tree model loaded successfully")
print(f"  Sample predictions: {test_pred_dt}")

print("\nAll models exported and verified successfully! ✅")


## 7. Summary

### Model Performance Summary:
- **Logistic Regression**: Good interpretability, linear relationships
- **Decision Tree**: Non-linear relationships, feature importance insights

### Next Steps:
1. Models are exported and ready for deployment
2. Integrate models into FastAPI backend
3. Create frontend interface for loan approval prediction
4. Deploy application to production

### Files Created:
- `Backend/model/loan_approval_lr.pkl` - Logistic Regression model
- `Backend/model/loan_approval_dt.pkl` - Decision Tree model
- `Backend/model/model_info.json` - Model metadata
